# GDP성장률, 인구증가율, 환율, 유가 변수 생성

In [188]:
import os
import pickle
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

import warnings
warnings.filterwarnings('ignore')

In [189]:
with open('../processed/export_GDP.pkl', 'rb') as f:
    export_GDP = pickle.load(f)
export_GDP = export_GDP.query('year<2023')

GDP = pd.read_csv('../data/GDP_ori.csv')    
population = pd.read_csv('../data/world_population.csv', skiprows=3, header=1)
fx_rate = pd.read_csv('../data/기간별평균환율.csv')

In [190]:
export_GDP

,year,country,품목코드,수출금액,GDP,preference
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07
...,...,...,...,...,...,...
129303,2022,Zimbabwe,90.0,299.0,NaN,NaN
129304,2022,Zimbabwe,94.0,21.0,NaN,NaN
129305,2022,Zimbabwe,95.0,2.0,NaN,NaN
129306,2022,Zimbabwe,96.0,0.0,NaN,NaN


In [191]:
export_GDP.shape, population.shape, fx_rate.shape

((129308, 6), (268, 67), (48, 12))

In [192]:
display(export_GDP.head(3))
display(population.head(3))
display(fx_rate.head(3))

,year,country,품목코드,수출금액,GDP,preference
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,...,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,...,567892149.0,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720839314.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,...,31541209.0,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0


,currency,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,USD,"1,126.43","1,094.97","1,053.30","1,132.10","1,161.11","1,130.61","1,101.08","1,166.51","1,179.90","1,145.07","1,293.68"
1,JPY,"1,412.96","1,123.73",996.37,935.06,"1,069.31","1,008.29",997.07,"1,070.63","1,105.36","1,041.92",985.24
2,EUR,"1,447.58","1,453.47","1,398.81","1,255.78","1,284.10","1,276.44","1,299.65","1,305.86","1,346.11","1,353.40","1,359.46"


### GDP성장률 파생변수 추가

In [193]:
export_GDP.head(2)

,year,country,품목코드,수출금액,GDP,preference
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00


In [194]:
# 나라별 연도별 GDP
cntry_year_GDP = export_GDP[['country','year','GDP']].drop_duplicates().pivot(index='country',columns='year',values='GDP')
cntry_year_GDP

year,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country,,,,,,,,,,,
Afghanistan,2.020357e+07,2.056449e+07,2.055058e+07,1.999816e+07,1.801956e+07,1.889635e+07,1.841885e+07,1.890449e+07,2.014344e+07,1.478686e+07,NaN
Albania,1.231983e+07,1.277622e+07,1.322815e+07,1.138685e+07,1.186120e+07,1.301969e+07,1.515643e+07,1.540183e+07,1.513187e+07,1.825579e+07,NaN
Algeria,2.090000e+08,2.100000e+08,2.140000e+08,1.660000e+08,1.600000e+08,1.700000e+08,1.750000e+08,1.720000e+08,1.450000e+08,1.630000e+08,NaN
American Samoa,6.400000e+05,6.380000e+05,6.430000e+05,6.730000e+05,6.710000e+05,6.120000e+05,6.390000e+05,6.470000e+05,7.160000e+05,7.090000e+05,NaN
Andorra,3.188809e+06,3.193704e+06,3.271808e+06,2.789870e+06,2.896679e+06,3.000181e+06,3.218316e+06,3.155065e+06,2.891022e+06,3.330282e+06,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,3.810000e+08,3.710000e+08,4.820000e+08,1.254490e+08,1.129150e+08,1.158830e+08,1.155883e+09,7.300000e+06,4.379000e+07,5.715000e+07,NaN
Vietnam,1.960000e+08,2.140000e+08,2.330000e+08,2.390000e+08,2.570000e+08,2.810000e+08,3.100000e+08,3.340000e+08,3.470000e+08,3.660000e+08,NaN
Yemen,3.540132e+07,4.041523e+07,4.322859e+07,4.244450e+07,3.131783e+07,2.684223e+07,2.160616e+07,2.189000e+07,1.884000e+07,2.106000e+07,NaN


In [195]:
display(GDP.head(2))
display(cntry_year_GDP.head(2))

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09,3.126019e+09,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129152e+10,2.180944e+10,2.370806e+10,2.821128e+10,2.611994e+10,2.968348e+10,...,9.830000e+11,1.000000e+12,9.230000e+11,8.900000e+11,1.030000e+12,1.020000e+12,1.010000e+12,9.340000e+11,1.090000e+12,NaN


year,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country,,,,,,,,,,,
Afghanistan,2.020357e+07,2.056449e+07,2.055058e+07,1.999816e+07,1.801956e+07,1.889635e+07,18418848.30,1.890449e+07,2.014344e+07,1.478686e+07,NaN
Albania,1.231983e+07,1.277622e+07,1.322815e+07,1.138685e+07,1.186120e+07,1.301969e+07,15156432.31,1.540183e+07,1.513187e+07,1.825579e+07,NaN


In [196]:
cntry_year_GDP.isna().sum()

year
2012      0
2013      0
2014      2
2015      2
2016      2
2017      2
2018      2
2019      0
2020      0
2021      0
2022    205
dtype: int64

In [197]:
# 결측치 채우기
failed = []
for i in range(len(cntry_year_GDP)):
    idx = cntry_year_GDP.index[i] # 국가명
    if cntry_year_GDP.loc[idx].isna().sum()>0: # 결측치가 있으면
        cols = cntry_year_GDP.loc[idx][cntry_year_GDP.loc[idx].isna()].index # 결측치가 있는 연도
        for col in cols:
            try:
                cntry_year_GDP.loc[idx, col] = GDP[GDP['Country Name']==idx][str(col)].values[0] # GDP에서 해당 국가, 연도의 GDP를 가져옴
            except:
                failed.append([idx, col])
failed

[['Guiana', 2022], ['Taiwan', 2022]]

In [198]:
cntry_year_GDP.isna().sum()

year
2012      0
2013      0
2014      0
2015      0
2016      0
2017      0
2018      0
2019      0
2020      0
2021      0
2022    205
dtype: int64

In [199]:
# 변화율 계산 함수
def get_growth_rate(x, cntry_year_value, value_col):
   if (x['year']-1) in cntry_year_value.columns:
      prev_year = cntry_year_value.loc[x['country'],(x['year']-1)]
      return (x[value_col] - prev_year)/prev_year

# GDP성장률 계산
export_GDP['GDP_growth'] = export_GDP.apply(lambda x: get_growth_rate(x, cntry_year_GDP, 'GDP'), axis=1)
export_GDP

,year,country,품목코드,수출금액,GDP,preference,GDP_growth
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN
...,...,...,...,...,...,...,...
129303,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN
129304,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN
129305,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN
129306,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN


In [200]:
export_GDP.query('year>2012').isna().sum()

year              0
country           0
품목코드              0
수출금액              0
GDP           12093
preference    12093
GDP_growth    12093
dtype: int64

### 인구증가율 파생변수 추가

In [201]:
population.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,...,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,...,567892149.0,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720839314.0


In [202]:
# 2012~2022년 인구 데이터만 추출
population_12_22 = population.drop(['Country Code','Indicator Name','Indicator Code'], axis=1)
for year in range(1960,2012):
    population_12_22 = population_12_22.drop(str(year), axis=1)

population_12_22 = population_12_22.rename(columns={'Country Name':'country'})
population_12_22 = population_12_22.set_index('country')
population_12_22.columns = population_12_22.columns.astype(int)
population_12_22.head(2)

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country,,,,,,,,,,,
Aruba,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0
Africa Eastern and Southern,552530654.0,567892149.0,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720839314.0


In [203]:
population_12_22.isna().sum()

2012    1
2013    1
2014    1
2015    1
2016    1
2017    1
2018    1
2019    1
2020    1
2021    1
2022    1
dtype: int64

In [204]:
# 'Not classified'는 원래 없는 국가이기 때문에 제거
population_12_22 = population_12_22.query('country != "Not classified"')
population_12_22.isna().sum()

2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    0
2019    0
2020    0
2021    0
2022    0
dtype: int64

In [205]:
population_12_22.head(2)

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country,,,,,,,,,,,
Aruba,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0
Africa Eastern and Southern,552530654.0,567892149.0,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720839314.0


In [206]:
# 인구 - 2012년~2021년 데이터만 추출
pop_12_21 = pd.DataFrame()
year_list = [i for i in range(2012, 2022)]
for year in year_list:
    pop_temp = population[['Country Name', str(year), str(year-1)]]
    pop_temp['year'] = year
    pop_temp.rename(columns={'Country Name':'country', str(year):'population'}, inplace=True)
    pop_temp = pop_temp[['country', 'year', 'population']]
    pop_12_21 = pd.concat([pop_12_21, pop_temp])
    
print('국가 수:', pop_12_21.country.nunique())
display(pop_12_21) 

국가 수: 268


,country,year,population
0,Aruba,2012,102112.0
1,Africa Eastern and Southern,2012,552530654.0
2,Afghanistan,2012,30466479.0
3,Africa Western and Central,2012,376797999.0
4,Angola,2012,25188292.0
...,...,...,...
263,South Africa,2021,59392255.0
264,Zambia,2021,19473125.0
265,Zimbabwe,2021,15993524.0
266,Taiwan,2021,23859912.0


In [207]:
# export_GDP와 pop_12_21 합치기
export_GDP_pop = export_GDP.copy()
export_GDP_pop = pd.merge(export_GDP_pop, pop_12_21, how='left', on=['country','year'])

print(export_GDP_pop.shape, export_GDP_pop.shape)
display(export_GDP_pop.head())

(129308, 8) (129308, 8)


,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0


In [208]:
cntry_year_GDP.columns

Index([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype='int64', name='year')

In [209]:
population_12_22.columns

Index([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype='int64')

In [210]:
# 인구증가율 계산
export_GDP_pop['pop_growth'] = export_GDP_pop.apply(lambda x: get_growth_rate(x, population_12_22, 'population'), axis=1)
export_GDP_pop

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN
...,...,...,...,...,...,...,...,...,...
129303,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN
129304,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN
129305,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN
129306,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN


In [211]:
export_GDP_pop.query('year>2012').isna().sum()

year              0
country           0
품목코드              0
수출금액              0
GDP           12093
preference    12093
GDP_growth    12093
population    12093
pop_growth    12093
dtype: int64

In [212]:
with open('../processed/export_GDP_pop.pkl', 'wb') as f:
    pickle.dump(export_GDP_pop, f)

### 환율 추가

In [213]:
display(fx_rate.head(2))
display(export_GDP_pop.head(2))

,currency,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,USD,"1,126.43","1,094.97","1,053.30","1,132.10","1,161.11","1,130.61","1,101.08","1,166.51","1,179.90","1,145.07","1,293.68"
1,JPY,"1,412.96","1,123.73",996.37,935.06,"1,069.31","1,008.29",997.07,"1,070.63","1,105.36","1,041.92",985.24


,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN


In [214]:
fx_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   currency  48 non-null     object
 1   2012      45 non-null     object
 2   2013      45 non-null     object
 3   2014      46 non-null     object
 4   2015      45 non-null     object
 5   2016      45 non-null     object
 6   2017      47 non-null     object
 7   2018      47 non-null     object
 8   2019      47 non-null     object
 9   2020      47 non-null     object
 10  2021      47 non-null     object
 11  2022      47 non-null     object
dtypes: object(12)
memory usage: 4.6+ KB


In [215]:
fx_rate.columns

Index(['currency', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022'],
      dtype='object')

In [216]:
fx_rate.currency.unique()

array(['USD', 'JPY', 'EUR', 'GBP', 'CAD', 'CHF', 'HKD', 'CNY', 'THB',
       'IDR', 'SEK', 'AUD', 'DKK', 'NOK', 'SAR', 'KWD', 'BHD', 'AED',
       'SGD', 'MYR', 'NZD', 'TWD', 'PHP', 'VND', 'PLN', 'RUB', 'ZAR',
       'INR', 'PKR', 'BDT', 'EGP', 'MXN', 'BRL', 'BND', 'ILS', 'JOD',
       'ARS', 'CZK', 'MNT', 'FJD', 'KHR', 'TRY', 'HUF', 'QAR', 'XAU',
       'XAG', 'KZT', 'NPR'], dtype=object)

In [217]:
fx_rate

,currency,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,USD,"1,126.43","1,094.97","1,053.30","1,132.10","1,161.11","1,130.61","1,101.08","1,166.51","1,179.90","1,145.07","1,293.68"
1,JPY,"1,412.96","1,123.73",996.37,935.06,"1,069.31","1,008.29",997.07,"1,070.63","1,105.36","1,041.92",985.24
2,EUR,"1,447.58","1,453.47","1,398.81","1,255.78","1,284.10","1,276.44","1,299.65","1,305.86","1,346.11","1,353.40","1,359.46"
3,GBP,"1,784.59","1,711.43","1,734.65","1,730.56","1,572.80","1,455.29","1,469.19","1,488.50","1,513.26","1,574.82","1,594.39"
4,CAD,"1,127.02","1,063.13",953.56,885.61,876.60,871.61,849.50,879.19,880.04,913.68,993.00
5,CHF,"1,201.01","1,180.84","1,151.46","1,175.96","1,178.11","1,148.56","1,125.04","1,174.19","1,257.49","1,252.09","1,353.99"
6,HKD,145.21,141.16,135.83,146.03,149.59,145.10,140.48,148.89,152.13,147.31,165.20
7,CNY,178.52,178.11,170.95,179.52,174.43,167.47,166.48,168.68,170.90,177.56,191.73
8,THB,36.24,35.69,32.43,33.05,32.89,33.33,34.05,37.61,37.74,35.82,36.88
9,IDR,12.04,10.58,8.89,8.46,8.74,8.45,7.74,8.26,8.15,8.02,8.71


In [218]:
year_cols = fx_rate.columns[1:]
for year in year_cols: 
    fx_rate[year] = fx_rate[year].apply(lambda x: x.replace(',','') if type(x)==str else x).astype(float)

In [219]:
currencies = ['USD','JPY','EUR','CNY']
for currency in currencies:
    export_GDP_pop[currency] = np.nan
export_GDP_pop.head(3) 

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,NaN,NaN,NaN,NaN
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,NaN,NaN,NaN,NaN
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,NaN,NaN,NaN,NaN


In [220]:
def get_fx_rate(x, currency):
    year = x['year']
    return fx_rate.loc[fx_rate.currency==currency, str(year)].values[0]  # 해당 연도의 통화 환율 가져오기

for currency in currencies:
    export_GDP_pop[currency] = export_GDP_pop.progress_apply(lambda x: get_fx_rate(x, currency), axis=1)
    
export_GDP_pop.sample(5)

100%|██████████| 129308/129308 [00:09<00:00, 13539.87it/s]


,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY
102329,2020,Saudi Arabia,65.0,73.0,7.030000e+08,1.038407e-07,-0.125622,35997107.0,0.004738,1179.90,1105.36,1346.11,170.90
64000,2017,Mauritius,39.0,2650.0,1.371351e+07,1.932401e-04,0.088879,1264887.0,0.000902,1130.61,1008.29,1276.44,167.47
103903,2020,Tunisia,38.0,1318.0,4.253783e+07,3.098418e-05,0.015075,12161723.0,0.009329,1179.90,1105.36,1346.11,170.90
119823,2022,Democratic Republic of the Congo,74.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73
81196,2019,Angola,72.0,2.0,6.930911e+07,2.885624e-08,-0.109057,32353588.0,0.034536,1166.51,1070.63,1305.86,168.68


In [221]:
# 선호도(해당 국가의 GDP 대비 한국품목 수입액)이 1을 넘기는 것은 이상치로 판단하여 제거
export_GDP_pop.query('preference>1').country.unique()

array(['Liberia', 'Marshall Islands'], dtype=object)

In [222]:
export_GDP_pop.country.nunique()

205

In [223]:
export_GDP_pop[~export_GDP_pop['country'].isin(['Liberia', 'Marshall Islands'])].country.nunique()

203

In [224]:
export_GDP_pop = export_GDP_pop[~export_GDP_pop['country'].isin(['Liberia', 'Marshall Islands'])]
export_GDP_pop.reset_index(drop=True, inplace=True)
export_GDP_pop

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73


In [225]:
# 저장
with open('../processed/export_GDP_pop_fx.pkl', 'wb') as f:
    pickle.dump(export_GDP_pop, f)

### 유가 추가

In [226]:
wti = pd.read_csv('../data/WTI.csv')

In [227]:
export_GDP_pop.shape, wti.shape

((128393, 13), (11, 2))

In [228]:
export_GDP_pop_fx_wti = pd.merge(export_GDP_pop, wti, how='left', on='year')
print(export_GDP_pop_fx_wti.shape)
display(export_GDP_pop_fx_wti)

(128393, 14)


,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000


In [229]:
# 저장
with open('../processed/export_GDP_pop_fx_wti.pkl', 'wb') as f:
    pickle.dump(export_GDP_pop_fx_wti, f)

### 국가명 인코딩

In [230]:
country_to_idx = {v:k for k,v in enumerate(export_GDP_pop['country'].unique())}
idx_to_country = {v:k for k,v in country_to_idx.items()}

print(country_to_idx)
print(idx_to_country)

{'Afghanistan': 0, 'Albania': 1, 'Algeria': 2, 'American Samoa': 3, 'Andorra': 4, 'Angola': 5, 'Antigua and Barbuda': 6, 'Argentina': 7, 'Armenia': 8, 'Aruba': 9, 'Australia': 10, 'Austria': 11, 'Azerbaijan': 12, 'Bahamas': 13, 'Bahrain': 14, 'Bangladesh': 15, 'Barbados': 16, 'Belarus': 17, 'Belgium': 18, 'Belize': 19, 'Benin': 20, 'Bermuda': 21, 'Bhutan': 22, 'Bolivia': 23, 'Bosnia and Herzegovina': 24, 'Botswana': 25, 'Brazil': 26, 'Brunei': 27, 'Bulgaria': 28, 'Burkina Faso': 29, 'Burundi': 30, 'Cambodia': 31, 'Cameroon': 32, 'Canada': 33, 'Cape Verde': 34, 'Cayman Islands': 35, 'Central African Republic': 36, 'Chad': 37, 'Chile': 38, 'China': 39, 'Colombia': 40, 'Comoros': 41, 'Congo': 42, 'Costa Rica': 43, 'Croatia': 44, 'Cuba': 45, 'Curacao': 46, 'Cyprus': 47, 'Czech Republic': 48, 'Democratic Republic of the Congo': 49, 'Denmark': 50, 'Djibouti': 51, 'Dominica': 52, 'Dominican Republic': 53, 'Ecuador': 54, 'Egypt': 55, 'El Salvador': 56, 'Equatorial Guinea': 57, 'Estonia': 58, '

In [231]:
with open('../processed/country_to_idx.pkl', 'wb') as f:
    pickle.dump(country_to_idx, f)

with open('../processed/idx_to_country.pkl', 'wb') as f:
    pickle.dump(idx_to_country, f)